In [ ]:
%pip install sqlalchemy==1.3.9

In [ ]:
%pip install ipython-sql
%pip install ipython-sql prettytable

In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'
con = sqlite3.connect('my_data1.db')
cur = con.cursor()

In [ ]:
%pip install -q pandas

In [ ]:
%sql sqlite:///my_data1.db

In [ ]:
import pandas as pd
df = pd.read_csv('Spacex.csv')
df.to_sql('SPACEXTBL', con, if_exists = 'replace', index=False, method='multi')

In [ ]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

In [ ]:
%%sql
SELECT DISTINCT Launch_Site FROM SPACEXTABLE

In [ ]:
%%sql
    
SELECT * FROM SPACEXTABLE
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5

In [ ]:
%%sql
    
SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PAYLOAD_MASS_KG FROM SPACEXTABLE
WHERE Customer LIKE 'NASA%'

In [ ]:
%%sql
    
SELECT ROUND(AVG(PAYLOAD_MASS__KG_), 2) AS AVG_PAYLOAD_MASS_KG FROM SPACEXTABLE
WHERE Booster_Version LIKE 'F9 v1.1%'

In [ ]:
%%sql
SELECT MIN(Date) 
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE '%Success%' 
  AND Landing_Outcome LIKE '%ground pad%';

In [ ]:
%%sql
SELECT Booster_Version 
FROM SPACEXTABLE
WHERE Landing_Outcome LIKE '%Success (drone ship)%' 
  AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000

In [ ]:
%%sql
SELECT 
  CASE 
    WHEN Mission_Outcome LIKE '%Success%' THEN 'Success'
    WHEN Mission_Outcome LIKE '%Failure%' THEN 'Failure'
    ELSE 'Other'
  END AS OutcomeType,
  COUNT(*) AS Total
FROM SPACEXTABLE
GROUP BY OutcomeType;

In [ ]:
%%sql
SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTABLE
    )

In [ ]:
%%sql
SELECT 
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE
WHERE 
    Landing_Outcome LIKE '%Failure%' 
    AND Landing_Outcome LIKE '%drone ship%'
    AND substr(Date, 1, 4) = '2015';

In [ ]:
%%sql
SELECT 
    Landing_Outcome, 
    COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
WHERE 
    Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;


---

# 🚀 SpaceX Mission SQL Findings Summary

## 1. Unique Launch Sites

**Query:** Display the names of the unique launch sites in the space mission.

| Launch_Site  |
| ------------ |
| CCAFS LC-40  |
| VAFB SLC-4E  |
| KSC LC-39A   |
| CCAFS SLC-40 |

---

## 2. Launch Sites Beginning with “CCA”

**Query:** Display 5 records where launch sites begin with the string `'CCA'`.

| Date       | Time (UTC) | Booster_Version | Launch_Site | Payload                                                       | PAYLOAD_MASS__KG_ | Orbit     | Customer        | Mission_Outcome | Landing_Outcome     |
| ---------- | ---------- | --------------- | ----------- | ------------------------------------------------------------- | ----------------- | --------- | --------------- | --------------- | ------------------- |
| 2010-06-04 | 18:45:00   | F9 v1.0 B0003   | CCAFS LC-40 | Dragon Spacecraft Qualification Unit                          | 0                 | LEO       | SpaceX          | Success         | Failure (parachute) |
| 2010-12-08 | 15:43:00   | F9 v1.0 B0004   | CCAFS LC-40 | Dragon demo flight C1, two CubeSats, barrel of Brouere cheese | 0                 | LEO (ISS) | NASA (COTS) NRO | Success         | Failure (parachute) |
| 2012-05-22 | 07:44:00   | F9 v1.0 B0005   | CCAFS LC-40 | Dragon demo flight C2                                         | 525               | LEO (ISS) | NASA (COTS)     | Success         | No attempt          |
| 2012-10-08 | 00:35:00   | F9 v1.0 B0006   | CCAFS LC-40 | SpaceX CRS-1                                                  | 500               | LEO (ISS) | NASA (CRS)      | Success         | No attempt          |
| 2013-03-01 | 15:10:00   | F9 v1.0 B0007   | CCAFS LC-40 | SpaceX CRS-2                                                  | 677               | LEO (ISS) | NASA (CRS)      | Success         | No attempt          |

---

## 3. Total Payload Mass by NASA (CRS)

**Query:** Display total payload mass carried by boosters launched by NASA (CRS).

| TOTAL_PAYLOAD_MASS_KG |
| --------------------- |
| 99980                 |

---

## 4. Average Payload Mass for F9 v1.1

**Query:** Display average payload mass carried by booster version F9 v1.1.

| AVG_PAYLOAD_MASS_KG |
| ------------------- |
| 2534.67             |

---

## 5. First Successful Ground Pad Landing

**Query:** List the date when the first successful landing outcome in ground pad was achieved.

| MIN(Date)  |
| ---------- |
| 2015-12-22 |

---

## 6. Boosters with Successful Drone Ship Landings and Payload Mass (4000–6000 KG)

**Query:** List the names of boosters with success in drone ship and payload between 4000 and 6000.

| Booster_Version |
| --------------- |
| F9 FT B1022     |
| F9 FT B1026     |
| F9 FT B1021.2   |
| F9 FT B1031.2   |

---

## 7. Mission Outcome Summary

**Query:** List total number of successful and failed mission outcomes.

| OutcomeType | Total |
| ----------- | ----- |
| Failure     | 1     |
| Success     | 100   |

---

## 8. Boosters Carrying Maximum Payload (Using Subquery)

**Query:** List all booster versions that carried the maximum payload mass.

| Booster_Version | PAYLOAD_MASS__KG_ |
| --------------- | ----------------- |
| F9 B5 B1048.4   | 15600             |
| F9 B5 B1049.4   | 15600             |
| F9 B5 B1051.3   | 15600             |
| F9 B5 B1056.4   | 15600             |
| F9 B5 B1048.5   | 15600             |
| F9 B5 B1051.4   | 15600             |
| F9 B5 B1049.5   | 15600             |
| F9 B5 B1060.2   | 15600             |
| F9 B5 B1058.3   | 15600             |
| F9 B5 B1051.6   | 15600             |
| F9 B5 B1060.3   | 15600             |
| F9 B5 B1049.7   | 15600             |

---

## 9. 2015 Drone Ship Failures by Month

**Query:** Display month names, failed drone ship landings, booster versions, and launch sites for 2015.

| Month | Landing_Outcome      | Booster_Version | Launch_Site |
| ----- | -------------------- | --------------- | ----------- |
| 01    | Failure (drone ship) | F9 v1.1 B1012   | CCAFS LC-40 |
| 04    | Failure (drone ship) | F9 v1.1 B1015   | CCAFS LC-40 |

---

## 10. Ranked Landing Outcomes (2010–2017)

**Query:** Rank the count of landing outcomes between 2010-06-04 and 2017-03-20, in descending order.

| Landing_Outcome        | Outcome_Count |
| ---------------------- | ------------- |
| No attempt             | 10            |
| Success (drone ship)   | 5             |
| Failure (drone ship)   | 5             |
| Success (ground pad)   | 3             |
| Controlled (ocean)     | 3             |
| Uncontrolled (ocean)   | 2             |
| Failure (parachute)    | 2             |
| Precluded (drone ship) | 1             |

---

✅ **Summary:**
These findings collectively summarize the **SpaceX Falcon 9 mission dataset** through SQL-based data retrieval and aggregation — highlighting unique launch sites, payload analysis, booster performance, landing success patterns, and temporal insights into mission outcomes.